In [6]:
# Examine Individual Player Dead Money Cases
print("=== INDIVIDUAL PLAYER DEAD MONEY ANALYSIS ===\n")

# Get players with dead money > 0
players_with_dm = cap_impact_df[cap_impact_df['dead_money_millions'] > 0].copy()
players_with_dm = players_with_dm.sort_values('dead_money_millions', ascending=False)

print(f"Total players with dead money: {len(players_with_dm)}")
print(f"Total dead money: ${players_with_dm['dead_money_millions'].sum():.2f}M\n")

# Top 15 individual cases
print("TOP 15 INDIVIDUAL DEAD MONEY CASES:")
print("-" * 80)
for idx, row in players_with_dm.head(15).iterrows():
    player_name = players_df[players_df['player_id'] == row['player_id']]['player_name'].iloc[0] if not players_df[players_df['player_id'] == row['player_id']].empty else 'Unknown'
    print(f"{player_name:30s} | {row['team']:3s} | {row['year']:4d} | ${row['dead_money_millions']:6.2f}M")

print("\n" + "=" * 80)
print("INTERPRETATION:")
print("=" * 80)
print("Each row = Cap charge for a player NO LONGER on the roster that year")
print("Example: Aaron Miller, TB, 2015, $35.9M means:")
print("  → TB paid $35.9M in cap space for Miller in 2015")
print("  → Miller was NOT on TB's roster in 2015")
print("  → This is 'ghost money' from a previous contract obligation")
print("\nWHAT WE DON'T SEE:")
print("  ✗ Original contract: total value, years, signing date")
print("  ✗ Termination reason: cut, injury, trade, retirement")
print("  ✗ Termination date: when did the event occur?")
print("  ✗ Future projections: what would 2016-2018 have paid?")
print("  ✗ Guarantee breakdown: how much was guaranteed vs. not")

=== INDIVIDUAL PLAYER DEAD MONEY ANALYSIS ===

Total players with dead money: 23
Total dead money: $195.71M

TOP 15 INDIVIDUAL DEAD MONEY CASES:
--------------------------------------------------------------------------------
Aaron Wallace                  | TEN | 2017 | $ 35.40M
Larry Ogunjobi                 | PIT | 2022 | $ 24.49M
Javon Wims                     | CHI | 2019 | $ 16.21M
Matt Lengel                    | CIN | 2018 | $ 15.33M
Patrick Peterson               | ARI | 2019 | $ 12.23M
Khalif Barnes                  | ARI | 2017 | $  9.87M
Jalen Camp                     | HOU | 2022 | $  9.64M
Jalen Davis                    | CIN | 2024 | $  9.05M
Todd Herremans                 | IND | 2015 | $  6.95M
Matt Henningsen                | DEN | 2022 | $  6.73M
Matt Lengel                    | IND | 2019 | $  6.40M
Jalen Thompson                 | ARI | 2024 | $  5.94M
David Bada                     | WAS | 2022 | $  4.78M
Larry Fitzgerald               | ARI | 2020 | $  4.44M
Khal

# NFL Dead Money Visualization (2015-2024)

Analyze and visualize top teams with highest dead money burden over time.

## Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Set styles
sns.set_style('whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 8)

## Load Data

In [2]:
# Load compensation data
data_dir = Path('../data/processed/compensation')

cap_impact_df = pd.read_csv(data_dir / 'mart_player_cap_impact.csv')
players_df = pd.read_csv(data_dir / 'dim_players.csv')
team_dead_money_df = pd.read_csv(data_dir / 'team_dead_money_by_year.csv')

print(f"Cap Impact Records: {len(cap_impact_df)}")
print(f"Team Dead Money Records: {len(team_dead_money_df)}")
print(f"\nYears: {sorted(team_dead_money_df['year'].unique())}")
print(f"Teams: {team_dead_money_df['team'].nunique()}")

Cap Impact Records: 22282
Team Dead Money Records: 320

Years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
Teams: 32


## Aggregate Dead Money by Team and Year

In [3]:
# Aggregate dead money by team and year
team_dead_money_pivot = team_dead_money_df.pivot(index='team', columns='year', values='dead_money_millions').fillna(0)

# Calculate cumulative dead money per team (across all years)
total_by_team = team_dead_money_df.groupby('team')['dead_money_millions'].sum().sort_values(ascending=False)

print("Top 10 Teams by Total Dead Money (2015-2024):")
print(total_by_team.head(10))

# Identify teams with most years having dead money
years_with_dm = team_dead_money_df[team_dead_money_df['dead_money_millions'] > 0].groupby('team').size().sort_values(ascending=False)
print("\nTeams with Most Years Having Dead Money:")
print(years_with_dm.head(10))

Top 10 Teams by Total Dead Money (2015-2024):
team
ARI    36.42
TEN    35.40
PIT    24.49
CIN    24.38
CHI    18.74
IND    13.35
WAS    10.68
HOU     9.64
DEN     6.73
BUF     4.25
Name: dead_money_millions, dtype: float64

Teams with Most Years Having Dead Money:
team
ARI    5
WAS    3
CHI    2
CIN    2
IND    2
ATL    1
BAL    1
BUF    1
DEN    1
HOU    1
dtype: int64


## Create Line Chart of Top Teams

In [4]:
# Get top 8 teams by total dead money
top_teams = total_by_team.head(8).index.tolist()

# Filter data for top teams
top_team_data = team_dead_money_df[team_dead_money_df['team'].isin(top_teams)]

# Create line chart with Plotly
fig = px.line(
    top_team_data,
    x='year',
    y='dead_money_millions',
    color='team',
    markers=True,
    title='Top 8 Teams: Dead Money Trends (2015-2024)',
    labels={'dead_money_millions': 'Dead Money ($M)', 'year': 'Year'},
    height=500,
    width=1000
)

fig.update_layout(
    hovermode='x unified',
    plot_bgcolor='rgba(240, 240, 240, 0.5)',
    font=dict(size=12),
    title_font_size=16
)

fig.show()

print("Top 8 Teams by Total Dead Money:")
for team in top_teams:
    total = total_by_team[team]
    print(f"  {team}: ${total:.2f}M")

Top 8 Teams by Total Dead Money:
  ARI: $36.42M
  TEN: $35.40M
  PIT: $24.49M
  CIN: $24.38M
  CHI: $18.74M
  IND: $13.35M
  WAS: $10.68M
  HOU: $9.64M


## Create Heatmap of Dead Money

In [5]:
# Create heatmap for teams with most dead money issues
top_15_teams = total_by_team.head(15).index.tolist()
heatmap_data = team_dead_money_df[team_dead_money_df['team'].isin(top_15_teams)].pivot(
    index='team',
    columns='year',
    values='dead_money_millions'
).fillna(0)

# Reorder teams by total
heatmap_data = heatmap_data.loc[top_15_teams]

# Create Plotly heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='YlOrRd',
    text=heatmap_data.values.round(1),
    texttemplate='$%{text:.1f}M',
    textfont={"size": 9},
    colorbar=dict(title='Dead Money ($M)')
))

fig.update_layout(
    title='Dead Money Heatmap: Top 15 Teams (2015-2024)',
    xaxis_title='Year',
    yaxis_title='Team',
    height=600,
    width=1000,
    font=dict(size=11)
)

fig.show()

print("Heatmap shows dead money burden by team and year (yellow/light = low, red/dark = high)")

Heatmap shows dead money burden by team and year (yellow/light = low, red/dark = high)


## Identify Repeat Offenders

In [ ]:
# Identify teams with consistent dead money issues
repeat_offenders = team_dead_money_df[team_dead_money_df['dead_money_millions'] > 0].groupby('team').agg({
    'dead_money_millions': ['sum', 'mean', 'count', 'max']
}).round(2)

repeat_offenders.columns = ['Total ($M)', 'Avg ($M)', 'Years with Dead Money', 'Max ($M)']
repeat_offenders = repeat_offenders.sort_values('Total ($M)', ascending=False)

print("REPEAT OFFENDERS: Teams with Most Dead Money Issues")
print("=" * 70)
print(repeat_offenders.head(15))

# Create bar chart
repeat_off_data = repeat_offenders.head(10).reset_index()
repeat_off_data.columns = ['team', 'total', 'avg', 'years', 'max']

fig = px.bar(
    repeat_off_data,
    x='team',
    y='total',
    color='years',
    title='Repeat Offenders: Total Dead Money & Years Affected (Top 10)',
    labels={'total': 'Total Dead Money ($M)', 'team': 'Team', 'years': 'Years\nwith DM'},
    height=500,
    width=1000,
    color_continuous_scale='RdYlGn_r'
)

fig.update_layout(
    plot_bgcolor='rgba(240, 240, 240, 0.5)',
    font=dict(size=12),
    title_font_size=14
)

fig.show()

## Understanding "Dead Money"

**What is Dead Money?**

Dead money is **guaranteed money still counting against a team's salary cap for a player who is NO LONGER on the roster**. It's the "ghost" of a contract that haunts the team's cap space.

**How it occurs:**
1. **Player is cut/released** → Remaining guaranteed money accelerates to current year
2. **Career-ending injury** → Guaranteed portions still owed but player can't play
3. **Trade** → Original team may retain partial cap hit
4. **Retirement with remaining guarantees** → Signing bonus acceleration

**In our data model:**
- `dead_money_millions` in `mart_player_cap_impact.csv` = the cap charge for players NO LONGER playing
- Sourced from `fact_player_contracts.csv` where `salary_type = 'dead_cap'`
- Currently loaded from sample CSV (`player_dead_money_sample.csv`)

**Critical limitation:** We see the *dead cap charge* in a given year, but we DON'T see:
- Original contract structure (total value, years, guarantees)
- When the terminating event occurred (injury date, cut date)
- What the player *would have been paid* in future years if the contract completed
- Remaining guaranteed vs. non-guaranteed breakdowns

**Example:** If we see Aaron Miller with $35.9M dead money in 2015:
- ✓ We know: TB paid $35.9M in cap space for a player NOT on the 2015 roster
- ✗ We don't know: Was this from a 5-year $100M deal? Did injury happen in 2014? What would 2016-2018 have paid?

In [ ]:
# Annual statistics
print("=== DEAD MONEY STATISTICS BY YEAR ===\n")

yearly_stats = []
for year in sorted(players_with_dm['year'].unique()):
    year_data = players_with_dm[players_with_dm['year'] == year]['dead_money_millions']
    yearly_stats.append({
        'year': year,
        'count': len(year_data),
        'total': year_data.sum(),
        'mean': year_data.mean(),
        'median': year_data.median(),
        'max': year_data.max()
    })

yearly_df = pd.DataFrame(yearly_stats)

print(yearly_df.to_string(index=False))

# Trend over time
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yearly_df['year'],
    y=yearly_df['total'],
    mode='lines+markers',
    name='Total Dead Money',
    line=dict(width=3)
))

fig.update_layout(
    title='Total NFL Dead Money by Year (Sample Data)',
    xaxis_title='Year',
    yaxis_title='Total Dead Money ($M)',
    height=400,
    width=1000,
    plot_bgcolor='rgba(240, 240, 240, 0.5)'
)

fig.show()

print("\nNote: These statistics are based on sample data merged from player_dead_money_sample.csv")
print("Real NFL-wide dead money trends would require complete Spotrac/OTC data")

In [ ]:
# Histogram: Overall dead money distribution
fig = px.histogram(
    players_with_dm,
    x='dead_money_millions',
    nbins=30,
    title='Dead Money Distribution: All Years (2015-2024)',
    labels={'dead_money_millions': 'Dead Money ($M)', 'count': 'Number of Cases'},
    height=400,
    width=1000
)

fig.update_layout(
    plot_bgcolor='rgba(240, 240, 240, 0.5)',
    showlegend=False
)

fig.show()

# Summary by quantiles
print("\nDead Money Quantiles (All Years):")
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
for q in quantiles:
    val = players_with_dm['dead_money_millions'].quantile(q)
    print(f"  {int(q*100)}th percentile: ${val:.2f}M")

print(f"\nInterpretation:")
print(f"  → 50% of dead money charges are under ${players_with_dm['dead_money_millions'].median():.2f}M")
print(f"  → 90% of dead money charges are under ${players_with_dm['dead_money_millions'].quantile(0.9):.2f}M")
print(f"  → Top 1% of cases are over ${players_with_dm['dead_money_millions'].quantile(0.99):.2f}M")

In [ ]:
# Distribution of dead money by year
fig = go.Figure()

for year in sorted(players_with_dm['year'].unique()):
    year_data = players_with_dm[players_with_dm['year'] == year]['dead_money_millions']
    
    fig.add_trace(go.Box(
        y=year_data,
        name=str(year),
        boxmean='sd'  # show mean and standard deviation
    ))

fig.update_layout(
    title='Dead Money Distribution by Year (2015-2024)',
    yaxis_title='Dead Money ($M)',
    xaxis_title='Year',
    height=500,
    width=1000,
    showlegend=False,
    plot_bgcolor='rgba(240, 240, 240, 0.5)'
)

fig.show()

print("Box plot shows distribution of dead money charges per player/team/year")
print("- Box = 25th to 75th percentile (middle 50% of data)")
print("- Line in box = median")
print("- Diamond = mean")
print("- Whiskers = range (excluding outliers)")
print("- Points = outliers (unusually high dead money charges)")

In [ ]:
# Analyze salary distribution
# Note: Our current data has salary_millions = 0 for most records (roster data without contract details)
# We can still analyze cap_hit distribution and dead money patterns

print("=== SALARY DISTRIBUTION OVERVIEW ===\n")

# Check what salary data we actually have
non_zero_salaries = cap_impact_df[cap_impact_df['salary_millions'] > 0]
non_zero_cap_hit = cap_impact_df[cap_impact_df['cap_hit_millions'] > 0]

print(f"Records with salary > 0: {len(non_zero_salaries):,} / {len(cap_impact_df):,}")
print(f"Records with cap_hit > 0: {len(non_zero_cap_hit):,} / {len(cap_impact_df):,}")
print(f"Records with dead_money > 0: {len(players_with_dm):,} / {len(cap_impact_df):,}")

if len(non_zero_salaries) > 0:
    print(f"\nSalary statistics (non-zero records):")
    print(f"  Mean: ${non_zero_salaries['salary_millions'].mean():.2f}M")
    print(f"  Median: ${non_zero_salaries['salary_millions'].median():.2f}M")
    print(f"  Min: ${non_zero_salaries['salary_millions'].min():.2f}M")
    print(f"  Max: ${non_zero_salaries['salary_millions'].max():.2f}M")
else:
    print("\n⚠️  Current dataset has salary_millions = 0 for all records")
    print("   This is because we only have roster data, not detailed contract breakdowns")
    print("   We can analyze dead_money distribution instead:")
    
    print(f"\nDead Money statistics:")
    print(f"  Mean: ${players_with_dm['dead_money_millions'].mean():.2f}M")
    print(f"  Median: ${players_with_dm['dead_money_millions'].median():.2f}M")
    print(f"  25th percentile: ${players_with_dm['dead_money_millions'].quantile(0.25):.2f}M")
    print(f"  75th percentile: ${players_with_dm['dead_money_millions'].quantile(0.75):.2f}M")
    print(f"  Min: ${players_with_dm['dead_money_millions'].min():.2f}M")
    print(f"  Max: ${players_with_dm['dead_money_millions'].max():.2f}M")

## NFL Salary Distribution Analysis